In [1]:
#!/Tsan/bin/python
# -*- coding: utf-8 -*-

In [34]:
# Libraries To Use
from __future__ import division 
import numpy as np
import pandas as pd
import statsmodels.api as sm
import os
from sklearn import linear_model
from datetime import datetime,time,date
import matplotlib.pyplot as plt
import seaborn as sns
import time
import os 
import h5py
import talib

In [3]:
# Import My own library for factor testing
from SingleFactorTest import factorFilterFunctions as ff
#from config import *

In [4]:
%load_ext line_profiler

In [5]:
path = ff.data_path # path

In [6]:
# --------------------------------------- Global Functions to def---------------------------------- #

In [7]:
# top functions to def
def simpleNormalize(narrowedData):
    dataWinsorized = narrowedData.copy()
    dataWinsorizedTrans = dataWinsorized.T
    MAD = 1.483*np.abs(dataWinsorizedTrans-dataWinsorizedTrans.median(skipna=True))
    return ((dataWinsorizedTrans - dataWinsorizedTrans.mean(axis=0, skipna=True))/dataWinsorizedTrans.std(axis=0, skipna=True)).T

In [8]:
# top function
def neutralizeFactor(normalizedFactorDF, normalizedLFCAPDF, datelist):
    factorNeutralized = pd.DataFrame(index=normalizedFactorDF.index, columns=normalizedFactorDF.columns, data=None, dtype = float)
    for date in datelist:
        LFCAPIndice = normalizedLFCAPDF.loc[date].dropna()
        factorIndice = normalizedFactorDF.loc[date].dropna()
        intersectionStocks = list(set(LFCAPIndice.index) & set(factorIndice.index))
        #dummy_Matrix = pd.get_dummies(IndustryDF.loc[date]).T.iloc[:-1]
        #dummy_Matrix = dummy_Matrix[intersectionStocks].append(LFCAPIndice.loc[intersectionStocks])
        try:
            result = sm. OLS(factorIndice.loc[intersectionStocks].T, LFCAPIndice.loc[intersectionStocks].T).fit()
            factorNeutralized.loc[date][intersectionStocks] = result.resid
        except:
            factorNeutralized.loc[date] = np.NaN
    return factorNeutralized

In [9]:
# --------------------------------------- Function Section End ---------------------------------- #

In [10]:
filenameFCAP =  'LZ_CN_STKA_VAL_A_FCAP.h5'
# Data prepared for Neuralization
FCAP1 = np.log10(ff.readh5data(path,filenameFCAP))
NormalizedFCAP = simpleNormalize(FCAP1 )

In [13]:
# first is to calculate forward adjusted pice
def calAdjustedPrice():
    # Adjusted factor
    AdjFacBackward=ff.readh5data(path,filenameAdjustFactor)
    #AdjFacBackward=AdjFacBackward[:]

    #PriceData to Adjust
    PriceToAdj=ff.readh5data(path,filenamePirce)
    #PriceToAdj=PriceToAdj[:]

    #Calculate
    print AdjFacBackward.shape
    print AdjFacBackward.max().shape
    AdjFacforward = AdjFacBackward / AdjFacBackward.max()
    adjustedPrice = (AdjFacforward * PriceToAdj).round(5)
    #adjustedPrice.index.name = 'LZ_GPA_DERI_AdjustedPriceForward_20-d' 
    adjustedPrice.index.name = 'Own_Factor_AdjustedPriceForward-1d'
    print adjustedPrice.index.name 
    adjustedPrice.to_csv(path+'Own_Factor_AdjustedPriceForward-1d.csv',na_rep='NaN',date_format='%Y%m%d')

In [ ]:
# --------------------------------------- Mass Index ---------------------------------- #

In [43]:
filenameClose = 'LZ_CN_STKA_QUOTE_TCLOSE.h5'
filenameHigh = 'LZ_CN_STKA_QUOTE_THIGH.h5'
filenameLow = 'LZ_CN_STKA_QUOTE_TLOW.h5' 

In [84]:
priceRange = ff.readh5data(path,filenameHigh) - ff.readh5data(path,filenameLow)

In [113]:
class CalOwnFactor(object):
    def __init__(self,path,factorName):
        self.factorName = factorName
        self.path = path
    def addData(self,filename):
        self.data = ff.read(self.path,filename)
    def saveData(self,data):
        ff.saveh5data(data,path,factorName)

In [92]:
singleEma = priceRange.copy()
singleNanCount =0
for name in priceRange.columns:
    try:
        singleEma[name] = talib.EMA(priceRange[name].values,timeperiod = 9)
    except Exception as e:
        #print str(e) 
        print 'too much NaN value for stock %s'%name
        assert str(e)== 'inputs are all NaN'
        singleEma [name] = np.NaN
        singleNanCount+=1
print singleNanCount

too much NaN value for stock 000003.SZ
too much NaN value for stock 000013.SZ
too much NaN value for stock 000015.SZ
too much NaN value for stock 600625.SH
too much NaN value for stock 600632.SH
too much NaN value for stock 000508.SZ
too much NaN value for stock 600646.SH
too much NaN value for stock 600669.SH
too much NaN value for stock 600670.SH
too much NaN value for stock 000542.SZ
too much NaN value for stock 600813.SH
too much NaN value for stock 000556.SZ
too much NaN value for stock 000047.SZ
too much NaN value for stock 600878.SH
too much NaN value for stock 000588.SZ
too much NaN value for stock 600709.SH
too much NaN value for stock 000405.SZ
too much NaN value for stock 000412.SZ
too much NaN value for stock 000621.SZ
too much NaN value for stock 000653.SZ
too much NaN value for stock 000660.SZ
too much NaN value for stock 000658.SZ
too much NaN value for stock 000675.SZ
too much NaN value for stock 000689.SZ
too much NaN value for stock 000730.SZ
too much NaN value for st

In [93]:
doubleEma = priceRange.copy()
doubleNanCount = 0
for name in priceRange.columns:
    try:
        doubleEma[name] = talib.EMA(singleEma[name].values,timeperiod = 9)
    except Exception as e:
        #print str(e) 
        print 'too much NaN value for stock %s'%name
        assert str(e)== 'inputs are all NaN'
        doubleEma [name] = np.NaN
        doubleNanCount+=1
print doubleNanCount

too much NaN value for stock 000003.SZ
too much NaN value for stock 000013.SZ
too much NaN value for stock 000015.SZ
too much NaN value for stock 600625.SH
too much NaN value for stock 600632.SH
too much NaN value for stock 000508.SZ
too much NaN value for stock 600646.SH
too much NaN value for stock 600669.SH
too much NaN value for stock 600670.SH
too much NaN value for stock 000542.SZ
too much NaN value for stock 600813.SH
too much NaN value for stock 000556.SZ
too much NaN value for stock 000047.SZ
too much NaN value for stock 600878.SH
too much NaN value for stock 000588.SZ
too much NaN value for stock 600709.SH
too much NaN value for stock 000405.SZ
too much NaN value for stock 000412.SZ
too much NaN value for stock 000621.SZ
too much NaN value for stock 000653.SZ
too much NaN value for stock 000660.SZ
too much NaN value for stock 000658.SZ
too much NaN value for stock 000675.SZ
too much NaN value for stock 600788.SH
too much NaN value for stock 000689.SZ
too much NaN value for st

In [94]:
OwnfactorMassIndex = (singleEma / doubleEma).rolling(window=25, min_periods=25).sum()
ff.saveh5data(OwnfactorMassIndex,path,'OwnfactorMassIndex')

,000411,600587,000410,600864,000012,600586,000413,000626,000627,000625,...,600803,000728,600836,600580,600360,000040,600148,600585,000919,000878
2010-01-04,23.8876,22.8582,23.0297,23.2718,24.7340,24.2356,24.2781,24.5732,23.3996,24.1715,...,24.4626,24.2027,22.6624,23.9340,25.6259,23.3795,25.2390,24.2874,23.6328,24.0169
2010-01-05,23.7007,22.7884,22.8491,23.1113,24.2228,23.8232,24.0291,24.2476,23.1557,24.0483,...,24.3219,24.2123,22.6991,24.0787,25.5368,23.1510,25.0839,24.2174,23.3859,23.9442
2010-01-06,23.5383,23.1641,22.8592,23.0202,23.8944,NaN,23.7885,24.0493,23.0714,23.9671,...,24.2842,24.2359,22.7177,24.2095,25.4206,23.1488,24.9708,24.0433,23.2468,24.0036
2010-01-07,23.5550,23.6142,22.9178,23.0959,23.9438,23.0140,23.6176,23.8945,23.0169,24.0766,...,24.2821,24.3757,22.7791,NaN,25.4004,23.2444,24.9711,24.0095,23.3006,24.2837
2010-01-08,23.5828,23.9669,23.0137,23.3039,23.9023,22.8056,23.5226,23.6524,23.0354,24.1707,...,24.2390,24.5093,22.8519,24.7671,25.6411,23.3427,25.0208,23.9979,23.0954,24.4794
2010-01-11,23.5159,24.0670,23.0081,23.3583,23.7817,22.6724,23.3493,23.2801,22.9248,24.1962,...,24.1644,24.8118,22.8947,25.0928,25.8031,23.3285,24.9710,24.1348,22.9089,24.5971
2010-01-12,23.7030,24.2715,23.0267,23.4283,23.7833,22.6955,23.2507,23.0220,22.9499,24.1683,...,24.1874,25.0595,23.0546,25.3267,25.7047,23.3737,24.9357,24.3457,22.8060,24.7655
2010-01-13,23.8855,24.8288,23.0097,23.5136,23.8195,22.7490,23.2202,22.8545,23.0653,24.1860,...,24.1623,25.3058,23.2762,25.5206,25.7633,23.3916,24.8854,24.5175,22.8975,24.8654
2010-01-14,23.9748,25.2707,22.9807,23.5853,23.8604,22.8338,23.2337,22.7283,23.1305,24.0936,...,24.1942,25.4339,23.4420,25.8422,25.7523,23.6131,24.8505,24.6474,23.0901,24.9512
2010-01-15,24.0937,25.7038,23.2040,23.6388,24.0679,23.0802,23.2481,22.6962,23.2932,24.0517,...,24.2387,25.4974,23.6437,26.3574,25.7763,23.7000,NaN,24.6944,23.3492,25.0131


In [33]:
priceRange.describe()

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,603367.SH,300705.SZ,300707.SZ,603103.SH,002903.SZ,002906.SZ,603110.SH,002905.SZ,300708.SZ,603499.SH
count,3103.000000,3103.000000,3103.000000,3103.000000,3103.000000,3103.000000,3103.000000,2283.000000,3103.000000,3103.000000,...,5.000000,3.000000,3.000000,1.00,0.0,0.0,0.0,0.0,0.0,0.0
mean,0.157625,0.195247,0.235927,0.412498,0.315956,0.142311,0.248076,0.232615,0.543397,0.472079,...,0.560000,0.830000,1.206667,3.71,NaN,NaN,NaN,NaN,NaN,NaN
std,0.190594,0.214515,0.202637,0.364595,0.372849,0.162294,0.445807,0.190507,0.768956,0.499961,...,1.252198,1.437602,2.090008,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,3.71,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.050000,0.070000,0.110000,0.190000,0.130000,0.050000,0.070000,0.120000,0.180000,0.170000,...,0.000000,0.000000,0.000000,3.71,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.100000,0.120000,0.190000,0.310000,0.210000,0.090000,0.150000,0.200000,0.310000,0.300000,...,0.000000,0.000000,0.000000,3.71,NaN,NaN,NaN,NaN,NaN,NaN
75%,0.200000,0.220000,0.310000,0.510000,0.360000,0.170000,0.270000,0.310000,0.540000,0.550000,...,0.000000,1.245000,1.810000,3.71,NaN,NaN,NaN,NaN,NaN,NaN
max,2.980000,1.860000,1.650000,3.420000,5.000000,1.660000,6.520000,2.750000,6.770000,3.850000,...,2.800000,2.490000,3.620000,3.71,NaN,NaN,NaN,NaN,NaN,NaN


,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300654.SZ,300706.SZ,603363.SH,603136.SH,603378.SH,002902.SZ,601086.SH,603367.SH,300705.SZ,300707.SZ
date,,,,,,,,,,,,,,,,,,,,,
2005-01-04,0.518899,2.703705,2.018497,2.370721,1.602684,2.384930,1.404676,0.196941,-0.724708,3.299799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,0.527706,2.718192,2.037542,2.366130,1.619067,2.397451,1.436456,0.200459,-0.724908,3.336583,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,0.532199,2.722659,2.055738,2.345439,1.619357,2.401566,1.444658,0.187567,-0.711742,3.344557,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,0.587700,2.735341,2.062844,2.355952,1.644424,2.408549,1.463452,0.056711,-0.724838,3.361178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,0.612146,2.752223,2.077923,2.366330,1.643585,2.409008,1.491330,0.158732,-0.732108,3.347895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,0.603898,2.745192,2.072478,2.386810,1.635029,2.408234,1.476776,0.099752,-0.689069,3.361503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,0.588602,2.730508,2.051775,2.376598,1.616307,2.402824,1.457308,0.115585,-0.704935,3.364290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,0.596470,2.734289,2.049037,2.408378,1.635194,2.401812,1.474649,0.102512,-0.678229,3.372721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,0.563664,2.733615,2.052473,2.403930,1.647812,2.398942,1.450911,0.055599,-0.692667,3.366404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


20050104

Timestamp('2005-01-04 00:00:00')

In [18]:
filename = 'facp'
ff.saveh5data(data,path,filename)

In [19]:
 NormalizedFCAP.index

DatetimeIndex(['2005-01-04', '2005-01-05', '2005-01-06', '2005-01-07',
               '2005-01-10', '2005-01-11', '2005-01-12', '2005-01-13',
               '2005-01-14', '2005-01-17',
               ...
               '2017-09-21', '2017-09-22', '2017-09-25', '2017-09-26',
               '2017-09-27', '2017-09-28', '2017-09-29', '2017-10-09',
               '2017-10-10', '2017-10-11'],
              dtype='datetime64[ns]', name=u'date', length=3102, freq=None)

In [21]:
ff.readh5data(path,filename + '.h5')

,000005.SZ,600601.SH,600602.SH,600651.SH,600652.SH,600653.SH,600654.SH,600656.SH,000004.SZ,000002.SZ,...,300654.SZ,300706.SZ,603363.SH,603136.SH,603378.SH,002902.SZ,601086.SH,603367.SH,300705.SZ,300707.SZ
date,,,,,,,,,,,,,,,,,,,,,
2005-01-04,0.518899,2.703705,2.018497,2.370721,1.602684,2.384930,1.404676,0.196941,-0.724708,3.299799,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-05,0.527706,2.718192,2.037542,2.366130,1.619067,2.397451,1.436456,0.200459,-0.724908,3.336583,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-06,0.532199,2.722659,2.055738,2.345439,1.619357,2.401566,1.444658,0.187567,-0.711742,3.344557,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-07,0.587700,2.735341,2.062844,2.355952,1.644424,2.408549,1.463452,0.056711,-0.724838,3.361178,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-10,0.612146,2.752223,2.077923,2.366330,1.643585,2.409008,1.491330,0.158732,-0.732108,3.347895,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-11,0.603898,2.745192,2.072478,2.386810,1.635029,2.408234,1.476776,0.099752,-0.689069,3.361503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-12,0.588602,2.730508,2.051775,2.376598,1.616307,2.402824,1.457308,0.115585,-0.704935,3.364290,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-13,0.596470,2.734289,2.049037,2.408378,1.635194,2.401812,1.474649,0.102512,-0.678229,3.372721,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2005-01-14,0.563664,2.733615,2.052473,2.403930,1.647812,2.398942,1.450911,0.055599,-0.692667,3.366404,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
